In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [16]:
orders_features = catalog.load("orders_features")
parameters = catalog.load("parameters")

2022-05-05 18:36:37,753 - kedro.io.data_catalog - INFO - Loading data from `orders_features` (PickleDataSet)...
2022-05-05 18:36:38,500 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...


## Feature importance

In [1]:
models_catboost = catalog.load("models_catboost")

2022-05-08 23:07:11,743 - kedro.io.data_catalog - INFO - Loading data from `models_catboost` (PickleDataSet)...


Order of Features in experiments - 
'weekdayOrdered', 'weekdayReceived', 'weDelivery', 'accOldSys', "color", "size", "userAgeImp""customerID", "manufacturerID", "salutation", "state", 'monthOrdered''price', 'discount', 'basket_price', 'basketNArts', 'durationDelivery', 'accAgeAtOrder', 'UserAgeOrder'

In [5]:
imps = {}
for experiment in models_catboost.keys():
    print(experiment, models_catboost[experiment].feature_importances_)
    imps[experiment] = pd.Series(models_catboost[experiment].feature_importances_)
    # (models_catboost[experiment].get_feature_names())

catboost_enc [17.40105106  3.44561146  6.5122208   0.39365653  0.12660463  3.45785026
  5.1645311   0.0701711  26.94141092  2.38274932  0.26282614  0.36219201
  1.66343074  3.31193607  1.23817653  6.12981824  3.54457467 14.33143702
  2.23815404  1.02159735]
mEstimate [ 3.42885371  6.37142187  0.77832295  0.35688068  3.9304525   5.692714
  0.12394899 23.53285636  6.97416121  0.27532355  0.59079215  2.85841797
  2.5671913   2.11253059  6.54049848  3.80791661 13.78372931  2.68287097
  1.69230601 11.8988108 ]
BetaLoo2D [ 3.93220469  6.25232046  0.74652879  0.31551947  3.54932055  5.25094872
  0.08642376 23.1987633   7.54468971  0.32557078  0.46571214  2.57134081
  3.43209359  1.79405242  6.54016093  3.59722098 13.83604495  2.3562168
  1.48092422 12.72394292]


In [18]:
# pd.DataFrame(imps)[["BetaLoo2D", "mEstimate", "catboost_enc"]]

In [19]:
"""from sklearn.preprocessing import OrdinalEncoder
def transform_input(data, parameters, test_trans=False, testset_value=1):
    ord_encode =  parameters["coldict"]["ord_encode"]
    if test_trans:
        testset_value +=1
    train = data[data["val_set"]<testset_value]
    test = data[data["val_set"]==testset_value]
    enc = OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value=np.nan) # )
    enc_train = enc.fit_transform(train[ord_encode])
    enc_test = enc.transform(test[ord_encode])
    data.loc[:, ord_encode] = np.nan
    data.loc[train.index, ord_encode] = enc_train
    data.loc[test.index, ord_encode] = enc_test
    for column in ord_encode:
        data[column].fillna(max(data[column]), inplace=True)
    catenc= parameters["coldict"]["cat_pass"] + parameters["coldict"]["cat_catboost"]
    data.loc[:, catenc] = data[catenc].apply(lambda x: pd.to_numeric(x))
    return data"""

'from sklearn.preprocessing import OrdinalEncoder\ndef transform_input(data, parameters, test_trans=False, testset_value=1):\n    ord_encode =  parameters["coldict"]["ord_encode"]\n    if test_trans:\n        testset_value +=1\n    train = data[data["val_set"]<testset_value]\n    test = data[data["val_set"]==testset_value]\n    enc = OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value=np.nan) # )\n    enc_train = enc.fit_transform(train[ord_encode])\n    enc_test = enc.transform(test[ord_encode])\n    data.loc[:, ord_encode] = np.nan\n    data.loc[train.index, ord_encode] = enc_train\n    data.loc[test.index, ord_encode] = enc_test\n    for column in ord_encode:\n        data[column].fillna(max(data[column]), inplace=True)\n    catenc= parameters["coldict"]["cat_pass"] + parameters["coldict"]["cat_catboost"]\n    data.loc[:, catenc] = data[catenc].apply(lambda x: pd.to_numeric(x))\n    return data'

In [10]:
test=False
testset_value=1

In [11]:
ord_encode =  parameters["coldict"]["ord_encode"]
if test:
    testset_value +=1
train = data[data["val_set"]<testset_value]
test = data[data["val_set"]==testset_value]
enc = OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value=np.nan) # )


In [14]:
enc

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=nan)

In [15]:
enc_train = enc.fit_transform(train[ord_encode])
enc_test = enc.transform(test[ord_encode])

In [ ]:
enc_train = enc.fit_transform(train[ord_encode])
enc_test = enc.transform(test[ord_encode])
data.loc[:, ord_encode] = np.nan
data.loc[train.index, ord_encode] = enc_train
data.loc[test.index, ord_encode] = enc_test
for column in ord_encode:
    data[column].fillna(max(data[column]), inplace=True)
catenc= parameters["coldict"]["cat_pass"] + parameters["coldict"]["cat_catboost"]
data.loc[:, catenc] = data[catenc].apply(lambda x: pd.to_numeric(x))
return data

In [37]:
results_cols = catalog.load("results_cols")
enc_cols = catalog.load("enc_cols")
res_df = pd.DataFrame(results_cols)
best_cols = []
for ind in res_df.iloc[:-1].index:
    best_cols += [res_df.loc[ind][res_df.loc[ind]==max(res_df.loc[ind])].index.to_series().iloc[0]]
    # print(ind, max(res_df.loc[ind]))

2022-05-03 19:21:24,959 - kedro.io.data_catalog - INFO - Loading data from `results_cols` (PickleDataSet)...
2022-05-03 19:21:24,971 - kedro.io.data_catalog - INFO - Loading data from `enc_cols` (PickleDataSet)...


In [38]:
for col in res_df.columns:
    res_df.loc["item_imp", col] = res_df.loc["feat_imp", col].loc[col] / res_df.loc["feat_imp", col].sum()

In [39]:
res_df.to_csv('/home/ubuntu/dmc/catboost_hyperparams_dmc14.csv', sep=';',  decimal=',')  

In [26]:
res_df.loc[['MCC_test', 'F1_test', 'Recall_test', 'Prec_test']].to_csv('/home/ubuntu/dmc/catboost_hyperparams_dmc14.csv', sep=';')  

In [27]:
res_df.loc["item_imp", :] = 0

In [8]:
res_df[enc_cols['m_smooth']].iloc[:4]

,10_m_smooth,20_m_smooth,50_m_smooth,100_m_smooth
MCC_test,0.348557,0.347929,0.348522,0.349223
F1_test,0.681968,0.680713,0.679662,0.682181
Recall_test,0.699229,0.695753,0.691638,0.699189
Prec_test,0.665538,0.66631,0.668095,0.665982


In [9]:
res_df[enc_cols['1D']].iloc[:4]

,10,20,50,100
MCC_test,0.334559,0.331253,0.335813,0.331036
F1_test,0.674401,0.674298,0.683493,0.674977
Recall_test,0.6898,0.693076,0.719485,0.695553
Prec_test,0.659674,0.656511,0.650931,0.655583


In [10]:
res_df[enc_cols['1D_base']].iloc[:4]

,10_base,20_base,50_base,100_base
MCC_test,0.349379,0.349401,0.350941,0.347204
F1_test,0.684963,0.682087,0.680628,0.679486
Recall_test,0.708498,0.69867,0.692117,0.692517
Prec_test,0.662941,0.666273,0.669514,0.666936


In [84]:
# new_f["enc"] = "BetaLoo"
# new_f.set_index("enc", append=True, inplace = True)
# mapper = new_f.columns.to_series().str.extract('(\d+)').to_dict()[0]
# new_f.rename(mapper, inplace = True, axis=1)

In [51]:
new_f_b = res_df[enc_cols['1D_base']]
# new_f_b["enc"] = "Beta"
# new_f_b.set_index("enc", append=True, inplace = True)

In [ ]:
[int(s) for s in str.split() if s.isdigit()]

In [52]:
mapper = new_f_b.columns.to_series().str.extract('(\d+)').to_dict()[0]
new_f_b.rename(mapper, inplace = True, axis=1)

<>:1: DeprecationWarning: invalid escape sequence \d
<>:1: DeprecationWarning: invalid escape sequence \d
/tmp/ipykernel_25083/4171924704.py:1: DeprecationWarning: invalid escape sequence \d
  mapper = new_f_b.columns.to_series().str.extract('(\d+)').to_dict()[0]
/tmp/ipykernel_25083/4171924704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_f_b.rename(mapper, inplace = True, axis=1)


In [92]:
pd.concat([, new_f_b.iloc[:4]], keys=['BetaLoo','Beta'])#.swaplevel(0,1,axis=1).sort_index(axis=1)

10        20        50       100        10  \
BetaLoo MCC_test     0.285979  0.287929  0.279059  0.278459       NaN   
        F1_test      0.674225  0.670812  0.665906  0.672716       NaN   
        Recall_test  0.746015  0.730553  0.722921  0.749451       NaN   
        Prec_test     0.61504  0.620104  0.617227  0.610235       NaN   
Beta    MCC_test          NaN       NaN       NaN       NaN  0.303391   
        F1_test           NaN       NaN       NaN       NaN   0.68336   
        Recall_test       NaN       NaN       NaN       NaN  0.760518   
        Prec_test         NaN       NaN       NaN       NaN  0.620417   

                           20        50       100  
BetaLoo MCC_test          NaN       NaN       NaN  
        F1_test           NaN       NaN       NaN  
        Recall_test       NaN       NaN       NaN  
        Prec_test         NaN       NaN       NaN  
Beta    MCC_test     0.299984  0.303987  0.302584  
        F1_test      0.682853  0.685025  0.682709  
        Recall_test  0.762995  0.766791   0.75888  
        Prec_test    0.617946  0.619017  0.620435

In [74]:
pd.concat([new_f.loc["MCC_test"], new_f_b.loc["MCC_test"]], axis=0)

10     0.285979
20     0.287929
50     0.279059
100    0.278459
10     0.303391
20     0.299984
50     0.303987
100    0.302584
Name: MCC_test, dtype: object

In [46]:
# generates double-cols
# https://stackoverflow.com/questions/45307296/join-merge-two-pandas-dataframes-and-use-columns-as-multiindex


## Runs to find best model catboost

In [20]:
models_catboost = catalog.load("models_catboost")

2022-05-08 23:58:25,222 - kedro.io.data_catalog - INFO - Loading data from `models_catboost` (PickleDataSet)...


In [29]:
resultdict_pytorch.keys()

dict_keys(['entity_embeddings', 'target_enc', 'BetaLoo2D'])

In [30]:
resultdict.keys()
keys = ['BetaLoo2D', 'target_enc', 'entity_embeddings']

In [26]:
resultdict = catalog.load("resultdict")
for experiment in keys:
    print(experiment)
    results = resultdict[experiment]
    config = results["id2config"][results["incumbent"]]["config"]
    res = results["res"]
    print(res.get_runs_by_id(res.get_incumbent_id())[-1]["info"])

2022-05-09 00:02:34,585 - kedro.io.data_catalog - INFO - Loading data from `resultdict` (PickleDataSet)...
BetaLoo2D
{'iterations': 507, 'F1_test': 0.6688887566216296, 'Recall_test': 0.6670491868149262, 'Prec_test': 0.6707385007162183, 'MCC_test_multiarts': 0.3306786336505524, 'Recall_test_multiarts': 0.6870004555431316, 'Prec_test_multiarts': 0.6725997405124878, 'depth': 9, 'l2_leaf_reg': 8, 'learning_rate': 0.04517453063297027}
mEstimate
{'iterations': 314, 'F1_test': 0.6686342477963751, 'Recall_test': 0.6678801788611451, 'Prec_test': 0.6693900214166733, 'MCC_test_multiarts': 0.3289561441008791, 'Recall_test_multiarts': 0.6880357808423406, 'Prec_test_multiarts': 0.6711913707429402, 'depth': 10, 'l2_leaf_reg': 26, 'learning_rate': 0.06536745805426632}
catboost_enc
{'iterations': 202, 'F1_test': 0.6703423108960374, 'Recall_test': 0.6687507419571841, 'Prec_test': 0.6719414735000596, 'MCC_test_multiarts': 0.3331931049784665, 'Recall_test_multiarts': 0.688656976021866, 'Prec_test_multiart

In [31]:
resultdict_pytorch = catalog.load("resultdict_pytorch")
keys
for experiment in keys:
    print(experiment)
    results = resultdict_pytorch[experiment]
    config = results["id2config"][results["incumbent"]]["config"]
    res = results["res"]
    print(res.get_runs_by_id(res.get_incumbent_id())[-1]["info"])

2022-05-09 00:06:28,073 - kedro.io.data_catalog - INFO - Loading data from `resultdict_pytorch` (PickleDataSet)...
BetaLoo2D
{'best_metrics': 0.2956935465335846, 'best_epoch': 20, 'cols': ['100_3_True_conf'], 'lr': 0.0007613321293553189, 'wd': 0.023117494554733373, 'emb_dropout': 0.02648720852830354, 'lin_layer_dropout': 0.05589227873863844, 'no_layers': 2, 'number of parameters': 33072835}
target_enc
{'best_metrics': 0.2679676115512848, 'best_epoch': 10, 'cols': [20], 'lr': 0.00015226341829186317, 'wd': 0.04468233132210749, 'emb_dropout': 0.06688412526636073, 'lin_layer_dropout': 0.06659223566174967, 'no_layers': 2, 'number of parameters': 33072835}
entity_embeddings
{'best_metrics': 0.23350465297698975, 'best_epoch': 9, 'cols': ['itemID'], 'lr': 0.00011207606211860574, 'wd': 0.09445645065743215, 'emb_dropout': 0.01834045098534338, 'lin_layer_dropout': 0.030424224295953775, 'no_layers': 1, 'number of parameters': 33416723}


In [ ]:
models_catboost[1]["mEstimate"].tree_count_

In [ ]:
models_catboost[1]["BetaLoo2D"].tree_count_

In [ ]:
# from hpbandster run

resultdict.keys()
# resultdict["baseline"]["id2config"][resultdict["incumbent"]]["config"]

In [ ]:
resultdict["baseline"]["res"].extract_HBS_learning_curves()

In [ ]:
resultdict["baseline"]["res"].get_incumbent_id()

In [ ]:
resultdict["baseline"].keys()#["info"]

In [ ]:
resultdict["baseline"]["all_runs"][0]["config_id"]

In [45]:
resultdict["baseline"]["all_runs"]

[config_id: (0, 0, 0)	budget: 9.000000	loss: 0.6652376486802705
 time_stamps: 0.006165742874145508 (submitted), 0.007113456726074219 (started), 128.21902918815613 (finished)
 info: {'F1_test': 0.6690903311679282, 'Recall_test': 0.666376478967987, 'Prec_test': 0.6718263783611266, 'MCC_test_multiarts': 0.3320304417405781, 'Recall_test_multiarts': 0.6868348034952582, 'Prec_test_multiarts': 0.673502538071066, 'depth': 6, 'l2_leaf_reg': 28, 'learning_rate': 0.23126286938981494},
 config_id: (0, 0, 0)	budget: 27.000000	loss: 0.6652376486802705
 time_stamps: 1342.4106793403625 (submitted), 1342.4107496738434 (started), 1465.5741028785706 (finished)
 info: {'F1_test': 0.6690903311679282, 'Recall_test': 0.666376478967987, 'Prec_test': 0.6718263783611266, 'MCC_test_multiarts': 0.3320304417405781, 'Recall_test_multiarts': 0.6868348034952582, 'Prec_test_multiarts': 0.673502538071066, 'depth': 6, 'l2_leaf_reg': 28, 'learning_rate': 0.23126286938981494},
 config_id: (0, 0, 0)	budget: 81.000000	loss:

In [49]:
resultdict["baseline"]["res"].get_runs_by_id(resultdict["baseline"]["res"].get_incumbent_id())[-1]["info"]

{'F1_test': 0.671495105778339,
 'Recall_test': 0.6732222705868387,
 'Prec_test': 0.6697767804417148,
 'MCC_test_multiarts': 0.3321804464129841,
 'Recall_test_multiarts': 0.692674038182797,
 'Prec_test_multiarts': 0.6717269076305221,
 'depth': 6,
 'l2_leaf_reg': 28,
 'learning_rate': 0.23697445375162673}

In [8]:
resultdict["baseline"]["id2config"][resultdict["baseline"]["incumbent"]]["config"]

{'depth': 6, 'l2_leaf_reg': 28, 'lr': 0.23697445375162673}

In [9]:
resultdict["mEstimate"]["id2config"][resultdict["baseline"]["incumbent"]]["config"]

{'depth': 10,
 'l2_leaf_reg': 22,
 'lr': 0.1145472944322216,
 'm': 10,
 'smooth': 'm_smooth'}

In [11]:
resultdict["BetaLoo2D"]["id2config"][resultdict["BetaLoo2D"]["incumbent"]]["config"]

{'depth': 8,
 'l2_leaf_reg': 20,
 'lr': 0.04259296073697268,
 'max_basket': 100,
 'n_min': 50,
 'sort': 'lift',
 'weight': False}

In [25]:
resultdict["BetaLoo2D"]["id2config"]

{(0,
  0,
  0): {'config': {'depth': 2,
   'l2_leaf_reg': 3,
   'lr': 0.2411583036197836,
   'max_basket': 1,
   'n_min': 10,
   'sort': 'lift',
   'weight': False}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  1): {'config': {'depth': 4,
   'l2_leaf_reg': 29,
   'lr': 0.2179075482831009,
   'max_basket': 3,
   'n_min': 50,
   'sort': 'conf',
   'weight': True}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  2): {'config': {'depth': 12,
   'l2_leaf_reg': 10,
   'lr': 0.16808606154514735,
   'max_basket': 1,
   'n_min': 10,
   'sort': 'conf',
   'weight': True}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  3): {'config': {'depth': 2,
   'l2_leaf_reg': 17,
   'lr': 0.26712118076578806,
   'max_basket': 5,
   'n_min': 50,
   'sort': 'lift',
   'weight': False}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  4): {'config': {'depth': 4,
   'l2_leaf_reg': 19,
   'lr': 0.11549055858976759,
   'max_basket': 3,
   'n_min': 20,
   'sort': 'lift',
   'w

In [4]:
resultdict.keys()

dict_keys(['baseline', 'mEstimate', 'BetaLoo2D'])

In [5]:
resultdict["baseline"]

{'res': <hpbandster.core.result.Result at 0x7f04ea9388e0>,
 'id2config': {(0,
   0,
   0): {'config': {'depth': 6,
    'l2_leaf_reg': 28,
    'lr': 0.23126286938981494}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   1): {'config': {'depth': 9,
    'l2_leaf_reg': 5,
    'lr': 0.25463626069833356}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   2): {'config': {'depth': 2,
    'l2_leaf_reg': 12,
    'lr': 0.14903219796791203}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   3): {'config': {'depth': 10,
    'l2_leaf_reg': 22,
    'lr': 0.15301090905897227}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   4): {'config': {'depth': 15,
    'l2_leaf_reg': 4,
    'lr': 0.22045669328745487}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   5): {'config': {'depth': 8,
    'l2_leaf_reg': 14,
    'lr': 0.2833705221385834}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   6): {'config': {'depth': 10,
    'l2_leaf_reg': 14,
    